In [1]:
import random
import math
from IPython.display import display
import pandas as pd
import pickle
import numpy as np
import time

In [2]:
def original_board(rows, columns):
    return np.zeros((rows, columns))

def val_turn(c4_board, column):
    return c4_board[len(c4_board)-1][column] == 0

def get_next_row(c4_board, column):
    return next((row for row in range(len(c4_board)) if c4_board[row][column] == 0), None)

def get_allowed_moves(c4_board):
    return [column for column in range(c4_board.shape[1]) if val_turn(c4_board, column)]

def get_next_position(c4_board, letter):
    rows, cols = c4_board.shape
    for row, row_vals in enumerate(c4_board):
        for col, col_val in enumerate(row_vals[:-3]):
            if all(elem == letter for elem in row_vals[col:col+4]):
                return row, col
        for col, col_vals in zip(range(cols), (c4_board[r][col] for r in range(row, min(row+4, rows)))):
            if all(elem == letter for elem in col_vals):
                return row, col
        for col, col_vals in enumerate(row_vals[:-3]):
            if row < rows-3 and col < cols-3:
                diag_vals = [c4_board[row+i][col+i] for i in range(4)]
                if all(elem == letter for elem in diag_vals):
                    return row, col
        for col, col_vals in enumerate(row_vals[:-3]):
            if row >= 3 and col < cols-3:
                diag_vals = [c4_board[row-i][col+i] for i in range(4)]
                if all(elem == letter for elem in diag_vals):
                    return row, col
    return -1, -1

def val_success(c4_board, letter):
    rows, columns = c4_board.shape
    for row in range(rows):
        for col in range(columns - 3):
            if all(c4_board[row][col + i] == letter for i in range(4)):
                return True

    for row in range(rows - 3):
        for col in range(columns):
            if all(c4_board[row + i][col] == letter for i in range(4)):
                return True

    for row in range(rows - 3):
        for col in range(columns - 3):
            if all(c4_board[row + i][col + i] == letter for i in range(4)):
                return True

    for row in range(3, rows):
        for col in range(columns - 3):
            if all(c4_board[row - i][col + i] == letter for i in range(4)):
                return True

    return False

def first_turn_toss():
    choices = [1,2]
    return random.choice(choices)

def val_final_turn(c4_board, SI_Agent_Letter, MinMax_Letter):
    return any(val_success(c4_board, letter) for letter in (SI_Agent_Letter, MinMax_Letter)) or not get_allowed_moves(c4_board)

def evaluate_score(c4_board, letter, SIAgentLetter, MinMaxLetter):
    score = 0
    OtherPlayerLetter = MinMaxLetter if letter == SIAgentLetter else SIAgentLetter
    rows, cols = c4_board.shape

    for i in range(rows):
        row_array = [int(x) for x in list(c4_board[i,:])]
        col_array = [int(x) for x in list(c4_board[:,i])]
        for j in range(cols-3):
            sub_row = row_array[j:j+4]
            sub_col = col_array[j:j+4]
            if sub_row.count(letter) == 4:
                score += 1000
            elif sub_row.count(letter) == 3 and sub_row.count(0) == 1:
                score += 100
            elif sub_row.count(letter) == 2 and sub_row.count(0) == 2:
                score += 10
            if sub_row.count(OtherPlayerLetter) == 3 and sub_row.count(0) == 1:
                score -= 10
            if sub_col.count(letter) == 4:
                score += 1000
            elif sub_col.count(letter) == 3 and sub_col.count(0) == 1:
                score += 100
            elif sub_col.count(letter) == 2 and sub_col.count(0) == 2:
                score += 10
            if sub_col.count(OtherPlayerLetter) == 3 and sub_col.count(0) == 1:
                score -= 10

    for i in range(rows-3):
        for j in range(cols-3):
            sub_diagonal1 = [c4_board[i+k][j+k] for k in range(4)]
            sub_diagonal2 = [c4_board[i+3-k][j+k] for k in range(4)]
            if sub_diagonal1.count(letter) == 4:
                score += 1000
            elif sub_diagonal1.count(letter) == 3 and sub_diagonal1.count(0) == 1:
                score += 100
            elif sub_diagonal1.count(letter) == 2 and sub_diagonal1.count(0) == 2:
                score += 10
            if sub_diagonal1.count(OtherPlayerLetter) == 3 and sub_diagonal1.count(0) == 1:
                score -= 10
            if sub_diagonal2.count(letter) == 4:
                score += 1000
            elif sub_diagonal2.count(letter) == 3 and sub_diagonal2.count(0) == 1:
                score += 100
            elif sub_diagonal2.count(letter) == 2 and sub_diagonal2.count(0) == 2:
                score += 10
            if sub_diagonal2.count(OtherPlayerLetter) == 3 and sub_diagonal2.count(0) == 1:
                score -= 10

    return score

def min_max_with_alpha_beta_pruning_and_depth(c4_board, current_depth, isMinMaxMove, MinMaxLetter, SIAgentLetter, alpha, beta):

    if val_final_turn(c4_board, SIAgentLetter, MinMaxLetter):

        if val_success(c4_board, MinMaxLetter):
            return (None, 10000000)

        elif val_success(c4_board, SIAgentLetter):
            return (None, -10000000)

        else:
            return (None, 0)

    if current_depth == 0:
        return (None, evaluate_score(c4_board, MinMaxLetter, SIAgentLetter, MinMaxLetter))

    possible_positions = get_allowed_moves(c4_board)

    if isMinMaxMove:
        optimisedScore = -math.inf
        optimisedPosition = random.choice(possible_positions)

        for position in possible_positions:
            random_row = get_next_row(c4_board, position)
            new_c4_board = c4_board.copy()
            new_c4_board[random_row][position] = MinMaxLetter
            current_minmax_score = min_max_with_alpha_beta_pruning_and_depth(new_c4_board, current_depth - 1, False, MinMaxLetter, SIAgentLetter, alpha, beta)[1]

            if current_minmax_score > optimisedScore:
                optimisedScore = current_minmax_score
                optimisedPosition = position

            alpha = max(optimisedScore, alpha)

            if alpha >= beta:
                break

        return optimisedPosition, optimisedScore

    else:
        optimisedScore = math.inf
        optimisedPosition = random.choice(possible_positions)

        for position in possible_positions:
            random_row = get_next_row(c4_board, position)
            new_c4_board = c4_board.copy()
            new_c4_board[random_row][position] = MinMaxLetter
            current_minmax_score = min_max_with_alpha_beta_pruning_and_depth(new_c4_board, current_depth - 1, True, MinMaxLetter, SIAgentLetter, alpha, beta)[1]

            if current_minmax_score < optimisedScore:
                optimisedScore = current_minmax_score
                optimisedPosition = position

            beta = min(beta, optimisedScore)

            if alpha >= beta:
                break

        return optimisedPosition, optimisedScore

def semi_intelligent_agent_move(c4_board, SIAgentLetter, MinMaxLetter):
    if val_final_turn(c4_board, SIAgentLetter, MinMaxLetter):
        siagent_row, siagent_col = get_next_position(c4_board, SIAgentLetter)
        if siagent_row != -1:
            return siagent_row, siagent_col
        else:
            minmax_row, minmax_col = get_next_position(c4_board, MinMaxLetter)
            if minmax_row != -1:
                return minmax_row, minmax_col
            else:
                possible_positions = get_allowed_moves(c4_board)
                random_row = get_next_row(c4_board, random.choice(possible_positions))
                random_col = random.choice(possible_positions)
                return random_row, random_col
    else:
        possible_positions = get_allowed_moves(c4_board)
        random_row = get_next_row(c4_board, random.choice(possible_positions))
        random_col = random.choice(possible_positions)

        return random_row, random_col

def print_c4_board(c4_board):
    
    symbols = {0: '.', 1: 'X', 2: 'O'}
    for row in reversed(c4_board):  
        print(' '.join(symbols[cell] for cell in row))
    print()


def play_connect4(SIAgent_plays_first, c4_board_shape, depth):
    c4_board = original_board(c4_board_shape[0], c4_board_shape[1])
    MinMaxLetter = 1
    SIAgentLetter = 2
    isGameOver = False
    gameWinner = ''
    
    while not isGameOver:
        print_c4_board(c4_board)  # Print the current state of the c4_board

        if SIAgent_plays_first:
            # Semi-Intelligent Agent makes a move
            si_chosen_row, si_chosen_column = semi_intelligent_agent_move(c4_board, SIAgentLetter, MinMaxLetter)

            if val_turn(c4_board, si_chosen_column):
                c4_board[si_chosen_row][si_chosen_column] = SIAgentLetter
                SIAgent_plays_first = False
                if val_success(c4_board, SIAgentLetter):
                    isGameOver = True
                    gameWinner = 'SIAgentWon'
            else:
                continue  
        else:
            # MinMax Agent makes a move
            minmax_chosen_column, _ = min_max_with_alpha_beta_pruning_and_depth(c4_board, depth, True, MinMaxLetter, SIAgentLetter, -math.inf, math.inf)

            if val_turn(c4_board, minmax_chosen_column):
                minmax_chosen_row = get_next_row(c4_board, minmax_chosen_column)
                c4_board[minmax_chosen_row][minmax_chosen_column] = MinMaxLetter
                SIAgent_plays_first = True
                if val_success(c4_board, MinMaxLetter):
                    isGameOver = True
                    gameWinner = 'MinMaxWon'
            else:
                continue  

        if not get_allowed_moves(c4_board):  # Checking if the c4_board is full for a draw
            isGameOver = True
            gameWinner = 'Draw'

    print_c4_board(c4_board) 
    return gameWinner

Random

In [5]:
games = 5
SIAgentWin = 0
MinMaxWin = 0
Draw = 0
c4_board_shape = (6, 7)
depth = 5

startTime = time.time()

for _ in (range(games)):
    SIAgent_plays_first = first_turn_toss() == 1
    try:
        winner = play_connect4(SIAgent_plays_first, c4_board_shape, depth)
    except:
        continue

    if winner == 'MinMaxWon':
        MinMaxWin += 1
    elif winner == 'SIAgentWon':
        SIAgentWin += 1
    else:
        Draw += 1

totalTime = time.time()-startTime

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . O .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . X . O .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . X . O O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X . . X . O O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X O . X . O O

. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
X O . X . O O

. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
X . O . . . .
X O . X . O O

. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
X . O . . . .
X O . X . O O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . .

In [6]:
print("\nGame Results:")
print("Total games played:", games)
print("MinMax Agent won:", MinMaxWin, "games")
print("Semi-Intelligent Agent won:", SIAgentWin, "games")
print("Draws:", Draw, "games")
print("Total game time:", totalTime, "seconds")
print("Average time per game:", totalTime / games, "seconds")


Game Results:
Total games played: 5
MinMax Agent won: 4 games
Semi-Intelligent Agent won: 1 games
Draws: 0 games
Total game time: 4.303584575653076 seconds
Average time per game: 0.8607169151306152 seconds


Game Results:
Total games played: 100
MinMax Agent won: 91 games
Semi-Intelligent Agent won: 9 games
Draws: 0 games
Total game time: 349.3670334815979 seconds
Average time per game: 3.493670334815979 seconds

Semi- Intelligent

In [7]:
import numpy as np
import random
import math
import time


games = 100
SIAgentWin = 0
MinMaxWin = 0
Draw = 0
c4_board_shape = (6, 7)
depth = 5

startTime = time.time()

for _ in range(games):
    SIAgent_plays_first = True  # SI Agent always plays first
    try:
        winner = play_connect4(SIAgent_plays_first, c4_board_shape, depth)
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

totalTime = time.time() - startTime



. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O X . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
. . O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
X . O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . O . . .
X . O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . O . . .
X X O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . O O . .
X X O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O X . O O . .
X X O X . . O

. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
O X . O O

In [8]:
# Printing results
print("\nGame Results:")
print("Total games played:", games)
print("MinMax Agent won:", MinMaxWin, "games")
print("Semi-Intelligent Agent won:", SIAgentWin, "games")
print("Draws:", Draw, "games")
print("Total game time:", totalTime, "seconds")
print("Average time per game:", totalTime / games, "seconds")



Game Results:
Total games played: 100
MinMax Agent won: 88 games
Semi-Intelligent Agent won: 12 games
Draws: 0 games
Total game time: 265.25339674949646 seconds
Average time per game: 2.6525339674949646 seconds


In [12]:
games = 100
SIAgentWin = 0
MinMaxWin = 0
Draw = 0
c4_board_shape = (6, 7)
depth = 5

startTime = time.time()

for _ in range(games):
    SIAgent_plays_first = False  # MinMax Agent always plays first
    try:
        winner = play_connect4(SIAgent_plays_first, c4_board_shape, depth)
        if winner == 'MinMaxWon':
            MinMaxWin += 1
        elif winner == 'SIAgentWon':
            SIAgentWin += 1
        else:
            Draw += 1
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

totalTime = time.time() - startTime



. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .
X . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O . . . . .
X . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O O . . . . .
X . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O . . . . .
X . . X . . O

. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O . . . . .
O . . X . . O

. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
O O . . . . .
O . . X . . O

. . . . . . .
. . . . . . .
X . . . . . .
X . . . . . .
O O . . . . .
O . . X . . O

. . . . . . .
X . . . . . .
X . . . . . .
X . . . . . .
O O . . .

In [13]:
# Printing results
print("\nGame Results:")
print("Total games played:", games)
print("MinMax Agent won:", MinMaxWin, "games")
print("Semi-Intelligent Agent won:", SIAgentWin, "games")
print("Draws:", Draw, "games")
print("Total game time:", totalTime, "seconds")
print("Average time per game:", totalTime / games, "seconds")


Game Results:
Total games played: 100
MinMax Agent won: 94 games
Semi-Intelligent Agent won: 6 games
Draws: 0 games
Total game time: 217.7063329219818 seconds
Average time per game: 2.177063329219818 seconds
